In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import (
    denumpyize,
    numpyize,
    candidate_to_live_config,
    calc_spans,
    analyze_fills,
    create_xk,
    round_dynamic,
    round_values,
)
from njit_funcs import *
from procedures import (
    dump_live_config,
    load_live_config,
    add_argparse_args,
    prepare_backtest_config,
    make_get_filepath,
)
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams["figure.figsize"] = [29, 18]
pd.set_option("precision", 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = "configs/backtest/default.hjson"
        self.exchange = "binance"
        self.symbol = "XMRUSDT"
        self.market_type = "futures"
        self.user = "binance_01"
        self.start_date = "2021-05-01"
        self.end_date = "2021-12-08"
        self.starting_balance = 10000.0
        self.starting_configs = ""
        self.base_dir = "backtests"


config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config["n_days"] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f"millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms")

In [ ]:
df = pd.DataFrame({"timestamp": timestamps, "qty": qtys, "price": prices}).set_index(
    "timestamp"
)
df.price.iloc[::100].plot(title="Coin Price", xlabel="Time", ylabel="Price")

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "config_name": "v5.3_test",
    "logging_level": 0,
    "long": {
        "enabled": True,
        "ema_span_min": 1440,  # in minutes
        "ema_span_max": 4320,
        "eprice_exp_base": 1.3,
        "eprice_pprice_diff": 0.01,
        "grid_span": 0.1,
        "initial_qty_pct": 0.01,
        "initial_eprice_ema_dist": -0.01,  # negative is closer; positive is further away
        "markup_range": 0.01,
        "max_n_entry_orders": 10,
        "min_markup": 0.01,
        "n_close_orders": 7,
        "wallet_exposure_limit": 0.5,
        "secondary_allocation": 0.5,
        "secondary_pprice_diff": 0.1,
        "auto_unstuck_wallet_exposure_threshold": 0.1,  # percentage of wallet_exposure_limit to trigger auto unstuck.
        # e.g. wallet_exposure_limit=0.06 and auto_unstuck_wallet_exposure_threshold=0.1: auto unstuck when wallet_exposure > 0.06 * (1 - 0.1) == 0.054
        "auto_unstuck_ema_dist": 0.02,
    },  # positive is further away; negative is closer
    "short": {
        "enabled": True,
        "ema_span_min": 1440,  # in minutes
        "ema_span_max": 4320,
        "eprice_exp_base": 1.3,
        "eprice_pprice_diff": 0.01,
        "grid_span": 0.3,
        "initial_qty_pct": 0.01,
        "initial_eprice_ema_dist": -0.01,  # negative is closer; positive is further away
        "markup_range": 0.01,
        "max_n_entry_orders": 10,
        "min_markup": 0.01,
        "n_close_orders": 7,
        "wallet_exposure_limit": 0.5,
        "secondary_allocation": 0.5,
        "secondary_pprice_diff": 0.1,
        "auto_unstuck_wallet_exposure_threshold": 0.1,
        "auto_unstuck_ema_dist": 0.02,
    },
}

# round for better human readability
hand_tuned = round_values(hand_tuned, 4)
config["starting_balance"] = 1000.0
config["latency_simulation_ms"] = 1000
config_to_test = {**config, **numpyize(hand_tuned)}
dump_live_config(config_to_test, make_get_filepath(f"tmp/hand_tuned.json"))
hand_tuned

In [ ]:
sts = time()
fills, stats = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f"seconds elapsed {elapsed:.4f}")
fdf, sdf, analysis = analyze_fills(fills, stats, config_to_test)
pprint.pprint(analysis)
fdf

In [ ]:
sdf

In [ ]:
sdf.balance.plot()
sdf.equity.plot(title="Balance and equity", xlabel="Time", ylabel="Balance")

In [ ]:
plot_fills(df, fdf, plot_whole_df=True, title="Overview Fills")

In [ ]:
fdf[fdf.psize > 0.0].psize.plot(
    title="Position size in terms of contracts",
    xlabel="Time",
    ylabel="Long Position size",
)
fdf[fdf.psize < 0.0].psize.plot(
    title="Position size in terms of contracts",
    xlabel="Time",
    ylabel="Short Position size",
)

In [ ]:
sdf.price.plot(title="Average entry price", xlabel="Time", ylabel="Price")
sdf.long_pprice.replace(0.0, np.nan).plot()
sdf.short_pprice.replace(0.0, np.nan).plot()

In [ ]:
lpprices = sdf[sdf.long_pprice != 0.0]
pa_distance_long = (lpprices.long_pprice - lpprices.price).abs() / lpprices.price
print(f"Mean price action distance long {pa_distance_long.mean():.6f}")
pa_distance_long.plot(
    title="Price action distance", xlabel="Time", ylabel="Price action distance"
)

In [ ]:
spprices = sdf[sdf.short_pprice != 0.0]
pa_distance_short = (spprices.short_pprice - spprices.price).abs() / spprices.price
print(f"Mean price action distance short {pa_distance_short.mean():.6f}")
pa_distance_short.plot(
    title="Price action distance", xlabel="Time", ylabel="Price action distance"
)

In [ ]:
balance = config_to_test["starting_balance"]
highest_bid = df.price.iloc[-1]

inverse = config_to_test["inverse"]
do_long = True
qty_step = config_to_test["qty_step"]
price_step = config_to_test["price_step"]
min_qty = config_to_test["min_qty"]
min_cost = config_to_test["min_cost"]
c_mult = config_to_test["c_mult"]

grid_span = config_to_test["long"]["grid_span"]
wallet_exposure_limit = config_to_test["long"]["wallet_exposure_limit"]
max_n_entry_orders = config_to_test["long"]["max_n_entry_orders"]
initial_qty_pct = config_to_test["long"]["initial_qty_pct"]
eprice_pprice_diff = config_to_test["long"]["eprice_pprice_diff"]
eprice_exp_base = config_to_test["long"]["eprice_exp_base"]
secondary_allocation = config_to_test["long"]["secondary_allocation"]
secondary_pprice_diff = config_to_test["long"]["secondary_pprice_diff"]

grid = calc_whole_long_entry_grid(
    balance,
    highest_bid,
    inverse,
    qty_step,
    price_step,
    min_qty,
    min_cost,
    c_mult,
    grid_span,
    wallet_exposure_limit,
    max_n_entry_orders,
    initial_qty_pct,
    eprice_pprice_diff,
    secondary_allocation,
    secondary_pprice_diff,
    eprice_exp_base=eprice_exp_base,
)
gdf = pd.DataFrame(grid, columns=["qty", "price", "psize", "pprice", "wallet_exposure"])

gdf.loc[:, "eprice_pprice_diff"] = abs(gdf.price - gdf.pprice) / gdf.price
gdf.loc[:, "ddown_factor"] = gdf.qty / gdf.psize.shift()
gdf.loc[:, "bkr_price"] = gdf.apply(
    lambda x: calc_bankruptcy_price(
        balance, x.psize, x.pprice, 0.0, 0.0, inverse, c_mult
    ),
    axis=1,
)
colors = "rbygcmk"
for i, e in enumerate(gdf.itertuples()):
    plt.plot([0, 2], [e.price, e.price], f"{colors[i%len(colors)]}-")
    plt.plot([1, 2], [e.pprice, e.pprice], f"{colors[i%len(colors)]}--")
gdf

In [ ]:
balance = config_to_test["starting_balance"]
lowest_ask = df.price.iloc[-1]

inverse = config_to_test["inverse"]
do_short = True
qty_step = config_to_test["qty_step"]
price_step = config_to_test["price_step"]
min_qty = config_to_test["min_qty"]
min_cost = config_to_test["min_cost"]
c_mult = config_to_test["c_mult"]

grid_span = config_to_test["short"]["grid_span"]
wallet_exposure_limit = config_to_test["short"]["wallet_exposure_limit"]
max_n_entry_orders = config_to_test["short"]["max_n_entry_orders"]
initial_qty_pct = config_to_test["short"]["initial_qty_pct"]
eprice_pprice_diff = config_to_test["short"]["eprice_pprice_diff"]
eprice_exp_base = config_to_test["short"]["eprice_exp_base"]
secondary_allocation = config_to_test["short"]["secondary_allocation"]
secondary_pprice_diff = config_to_test["short"]["secondary_pprice_diff"]

grid = calc_whole_short_entry_grid(
    balance,
    highest_bid,
    inverse,
    qty_step,
    price_step,
    min_qty,
    min_cost,
    c_mult,
    grid_span,
    wallet_exposure_limit,
    max_n_entry_orders,
    initial_qty_pct,
    eprice_pprice_diff,
    secondary_allocation,
    secondary_pprice_diff,
    eprice_exp_base=eprice_exp_base,
)
gdf = pd.DataFrame(grid, columns=["qty", "price", "psize", "pprice", "wallet_exposure"])

gdf.loc[:, "eprice_pprice_diff"] = abs(gdf.price - gdf.pprice) / gdf.price
gdf.loc[:, "ddown_factor"] = gdf.qty / gdf.psize.shift()
gdf.loc[:, "bkr_price"] = gdf.apply(
    lambda x: calc_bankruptcy_price(
        balance, x.psize, x.pprice, 0.0, 0.0, inverse, c_mult
    ),
    axis=1,
)
colors = "rbygcmk"
for i, e in enumerate(gdf.itertuples()):
    plt.plot([0, 2], [e.price, e.price], f"{colors[i%len(colors)]}-")
    plt.plot([1, 2], [e.pprice, e.pprice], f"{colors[i%len(colors)]}--")
gdf